# Final Report

Loizos Koutsantonis

## The Battle of Neighborhoods

### Load Libraries

In [30]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests


!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 


import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



### Credentials for Foursquare Database

In [31]:
CLIENT_ID = 'X4OG2IWXBES4VSVI4RLKT4E1E5GBZWES2WSAMKF5UVVPPXF0' # your Foursquare ID
CLIENT_SECRET = 'O5MC555BC0JXCFNIIINE3XSO4V15IL0AOVRXHQVBASDCWXXL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: X4OG2IWXBES4VSVI4RLKT4E1E5GBZWES2WSAMKF5UVVPPXF0
CLIENT_SECRET:O5MC555BC0JXCFNIIINE3XSO4V15IL0AOVRXHQVBASDCWXXL


### Function for gathering the location of venues from Foursquare

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=4000):
    LIMIT=1000
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Gather Neighborhoods locations in Toronto from Wikipedia

In [9]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url, 'lxml')

table = str(soup.find("table", { "class" : "wikitable sortable" }))

df =pd.read_html(table)[0]
df.columns = ['PostalCode', 'Borough', 'Neighborhood']
df.drop(df.index[0], inplace=True)
df = df[df['Borough']!="Not assigned"]
df.Neighborhood[df.Neighborhood=="Not assigned"] = df.Borough[df.Neighborhood=="Not assigned"]
df=df.groupby(['PostalCode']).agg({'Borough': 'first','Neighborhood':', '.join})
df.reset_index(inplace=True)
df.shape

url="http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv"
coordinates=pd.read_csv(url)
coordinates.columns = ['PostalCode', 'Latitude', 'Longitude']
df1 = pd.merge(df,coordinates, on="PostalCode")

df1 = df1[df1['Borough'].str.contains('Toronto')].reset_index(drop=True)
df1.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


#### Gather all venues in Toronto from Foursquare - Use the locations of Neighborhoods as inputs

In [33]:
venues = getNearbyVenues(names=df1['Neighborhood'],
                        latitudes=df1['Latitude'],
                        longitudes=df1['Longitude']
                                  )
venues.columns =["Neighborhood","Neighborhood Latitude", "Neighborhood Longitude", "Venue", "Latitude", "Longitude", "Category" ]
venues.head(10)

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Latitude,Longitude,Category
0,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
1,The Beaches,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater
2,The Beaches,43.676357,-79.293031,Ed's Real Scoop,43.672630,-79.287993,Ice Cream Shop
3,The Beaches,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery
4,The Beaches,43.676357,-79.293031,Bagels On Fire,43.672864,-79.286784,Bagel Shop
5,The Beaches,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
6,The Beaches,43.676357,-79.293031,Kew Gardens,43.669038,-79.298538,Park
7,The Beaches,43.676357,-79.293031,I'm On The Beach,43.670364,-79.284696,Beach
8,The Beaches,43.676357,-79.293031,The Remarkable Bean,43.672801,-79.287038,Coffee Shop
9,The Beaches,43.676357,-79.293031,Kew-Balmy Beach,43.667372,-79.295312,Beach


### Find the coffee shop, restaurants, bars and hotels in venues and create a new dataframe for each of them

In [34]:
coffee_shops = venues[venues['Category']=='Coffee Shop']
restaurants = venues[venues['Category'].str.contains('Restaurant') ==1]
bars = venues[(venues['Category'].str.contains('Bar') ==1) & (venues['Category'].str.contains('Barbershop')==0)]
hotels = venues[(venues['Category'].str.contains('Hotel') ==1) ]

In [35]:
(coffee_shops.head(10))

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Latitude,Longitude,Category
8,The Beaches,43.676357,-79.293031,The Remarkable Bean,43.672801,-79.287038,Coffee Shop
11,The Beaches,43.676357,-79.293031,Buds Coffee Bar,43.669375,-79.303218,Coffee Shop
54,The Beaches,43.676357,-79.293031,Circus Coffee House,43.685483,-79.315364,Coffee Shop
68,The Beaches,43.676357,-79.293031,Starbucks,43.668370,-79.308015,Coffee Shop
75,The Beaches,43.676357,-79.293031,Starbucks,43.682446,-79.327232,Coffee Shop
120,"The Danforth West, Riverdale",43.679557,-79.352188,Hailed Coffee,43.666900,-79.345432,Coffee Shop
140,"The Danforth West, Riverdale",43.679557,-79.352188,Merchants of Green Coffee,43.659986,-79.354299,Coffee Shop
150,"The Danforth West, Riverdale",43.679557,-79.352188,Te Aro,43.661373,-79.338577,Coffee Shop
154,"The Danforth West, Riverdale",43.679557,-79.352188,Dark Horse Espresso Bar,43.658498,-79.352356,Coffee Shop
169,"The Danforth West, Riverdale",43.679557,-79.352188,Sumach Espresso,43.658135,-79.359515,Coffee Shop


In [36]:
(restaurants.head(10))

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Latitude,Longitude,Category
0,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
15,The Beaches,43.676357,-79.293031,Delina Restaurant,43.668867,-79.305404,Middle Eastern Restaurant
19,The Beaches,43.676357,-79.293031,Veloute Bistro,43.672267,-79.289584,French Restaurant
22,The Beaches,43.676357,-79.293031,Budapest Restaurant,43.680946,-79.310110,Hungarian Restaurant
25,The Beaches,43.676357,-79.293031,Jatujak,43.688421,-79.270073,Thai Restaurant
26,The Beaches,43.676357,-79.293031,Udupi Palace,43.672480,-79.321275,Indian Restaurant
30,The Beaches,43.676357,-79.293031,Lake Inez,43.672520,-79.320712,Asian Restaurant
40,The Beaches,43.676357,-79.293031,The Wren,43.682467,-79.328079,American Restaurant
44,The Beaches,43.676357,-79.293031,Melanie's Bistro,43.684800,-79.317167,French Restaurant
47,The Beaches,43.676357,-79.293031,Maha's Fine Egyptian Cuisine,43.671758,-79.328444,African Restaurant


In [17]:
(bars.head(10))

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Latitude,Longitude,Category
46,The Beaches,43.676357,-79.293031,Relish Bar & Grill,43.686280,-79.310980,Bar
82,The Beaches,43.676357,-79.293031,Pinkerton Snack Bar,43.668900,-79.337309,Cocktail Bar
88,The Beaches,43.676357,-79.293031,The Only Cafe,43.680409,-79.337898,Beer Bar
89,The Beaches,43.676357,-79.293031,Hitch Bar,43.663250,-79.330649,Bar
92,The Beaches,43.676357,-79.293031,The Shore Leave,43.684259,-79.319474,Cocktail Bar
118,"The Danforth West, Riverdale",43.679557,-79.352188,The Only Cafe,43.680409,-79.337898,Beer Bar
128,"The Danforth West, Riverdale",43.679557,-79.352188,Pinkerton Snack Bar,43.668900,-79.337309,Cocktail Bar
173,"The Danforth West, Riverdale",43.679557,-79.352188,The Comrade,43.659346,-79.347932,Bar
193,"The Danforth West, Riverdale",43.679557,-79.352188,Greenhouse Juice Co,43.679101,-79.390686,Juice Bar
236,"The Beaches West, India Bazaar",43.668999,-79.315572,Pinkerton Snack Bar,43.668900,-79.337309,Cocktail Bar


In [18]:
(hotels.head(10))

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Latitude,Longitude,Category
145,"The Danforth West, Riverdale",43.679557,-79.352188,The Broadview Hotel,43.659072,-79.350074,Hotel
293,"The Beaches West, India Bazaar",43.668999,-79.315572,The Broadview Hotel,43.659072,-79.350074,Hotel
307,Studio District,43.659526,-79.340923,The Broadview Hotel,43.659072,-79.350074,Hotel
399,Studio District,43.659526,-79.340923,The Grand Hotel & Suites Toronto,43.656449,-79.374110,Hotel
830,"Moore Park, Summerhill East",43.689574,-79.383160,Four Seasons Hotel Toronto,43.671796,-79.389457,Hotel
929,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,Four Seasons Hotel Toronto,43.671796,-79.389457,Hotel
1012,Rosedale,43.679563,-79.377529,Four Seasons Hotel Toronto,43.671796,-79.389457,Hotel
1066,Rosedale,43.679563,-79.377529,The Grand Hotel & Suites Toronto,43.656449,-79.374110,Hotel
1091,Rosedale,43.679563,-79.377529,The Broadview Hotel,43.659072,-79.350074,Hotel
1118,"Cabbagetown, St. James Town",43.667967,-79.367675,The Grand Hotel & Suites Toronto,43.656449,-79.374110,Hotel


In [37]:
print(coffee_shops.shape)
print(restaurants.shape)
print(bars.shape)
print(hotels.shape)

(258, 7)
(909, 7)
(178, 7)
(73, 7)


#### Markk the venues of interest in the map of Toronto

In [147]:
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, neighborhood in zip(restaurants['Latitude'], restaurants['Longitude'], restaurants['Neighborhood']):
    label = neighborhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
for lat, lng, neighborhood in zip(coffee_shops['Latitude'], coffee_shops['Longitude'], coffee_shops['Neighborhood']):
    label = neighborhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
for lat, lng, neighborhood in zip(bars['Latitude'], bars['Longitude'], bars['Neighborhood']):
    label = neighborhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='black',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
for lat, lng, neighborhood in zip(hotels['Latitude'], hotels['Longitude'], hotels['Neighborhood']):
    label = neighborhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


### Concatanate the locations of the venues in a new dataframe - Find the central location and the standard deviation in location (x,y)

In [59]:
clients = pd.concat([coffee_shops,bars, restaurants, hotels])
data =clients[['Latitude', 'Longitude']].values
x0 =np.mean(data[:,0])
y0=np.mean(data[:,1])

### Perform a kmeans clustering to define the 5 clusters of venues (use the location features as inputs)

In [83]:
# set number of clusters
kclusters = 5
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(data)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:] 

clients['Labels'] = kmeans.labels_

### Mark the venues again on the map of Toronto - Use different colors to indicate the clusters

In [148]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for lat, lng, cluster in zip(clients['Latitude'], clients['Longitude'], clients['Labels']):
    label = cluster
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto

### Function returning the mean radius of each cluster

In [153]:
def mean_radius(cl,loc):
    num=0
    sum_r = 0
    for x,y in zip(cl['Latitude'],cl['Longitude']):
        r= np.sqrt(((x-loc[0])*110)**2+((y-loc[1])*110*np.cos(x))**2)
        
        sum_r = sum_r+r
        num=num+1
    mean_r = sum_r/num
   

    return mean_r,num

### Find the number of venues and the mean radius before clustering

In [154]:
cl=clients
loc=[x0,y0]
r,num_all=mean_radius(cl,loc)
print(loc,r, num_all)

[43.66419877433535, -79.39172079048484] 3.54458463779814 1418


### Find the number of venues and mean radius for each of the five clusters

In [155]:
locations =[]
radii=[]
num_per_cluster=[]
for i in range(kclusters):
    cl=clients[clients['Labels']==i]
    loc=kmeans.cluster_centers_[i]
    r,num=mean_radius(cl,loc)
    locations.append(loc)
    radii.append(r)
    num_per_cluster.append(num)
locations =np.array(locations)
raddi=np.array(radii)

### Calculate the density of Each Cluster

In [156]:
num_per_cluster

[182, 425, 231, 166, 414]

In [157]:
locations

array([[ 43.66962932, -79.33436077],
       [ 43.65410104, -79.37749776],
       [ 43.70438192, -79.39764063],
       [ 43.65665602, -79.44908094],
       [ 43.65278082, -79.40523539]])

In [165]:
3*np.array(radii)

array([5.24043761, 3.49749161, 4.1838968 , 5.87242464, 3.81402628])

In [169]:
area = (np.pi*(3*np.array(radii))**2)
area

array([ 86.27500277,  38.42936733,  54.99355576, 108.33898857,
        45.70010885])

In [170]:
(np.array(num_per_cluster))/area

array([ 2.1095334 , 11.0592505 ,  4.20049216,  1.53222771,  9.05905938])

### Mark each cluster on the map -Use variable radius for each cluster to indicate its radius

In [104]:
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, r, lbl in zip(locations[:,0], locations[:,1], radii, range(kclusters)):
    label = lbl
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=r*5000,
        popup=label,
        color=rainbow[lbl-1],
        fill=True,
        fill_color=rainbow[lbl-1],
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
    print(lat,lng,r)
map_toronto

43.66962932424312 -79.33436077076588 0.01648166812303524
43.65410103991324 -79.37749776146173 0.010916298968176388
43.70438191588358 -79.39764062641856 0.012821695898094041
43.656656023341846 -79.44908093635581 0.01836691409659674
43.65278082267702 -79.40523539098822 0.011907687961049171
